# December 10, 2024

https://adventofcode.com/2024/day/10

In [6]:
from collections import defaultdict

In [67]:
DEBUG = False
def dprint( *args ):
    if DEBUG:
        return print(*args)


In [3]:
test_str = f'''89010123
78121874
87430965
96549874
45678903
32019012
01329801
10456732'''
test_str = [ x.strip() for x in test_str.split("\n")]

In [60]:
fn = "../data/2024/10.txt"
with open(fn, "r") as file:
    text = file.readlines()
puzz_str = [line.strip() for line in text]

# Part 1

In [156]:
class TopoMap:
    def __init__( self, lines ):
        self.nr = len(lines)
        self.nc = len(lines[0])

        # key by height and list all levels
        self.levels = dict()

        # key by [row][col] and give height and the views available starting from there
        self.pos_info = dict()
        for height in range(0,10):
            self.levels[height] = list()

        # iterate over input and build out both structs
        for r, line in enumerate(lines):
            self.pos_info[r] = dict()
            for c, x in enumerate(line):
                self.pos_info[r][c] = { "height": int(x), "views": None, "trails": None }
                self.levels[int(x)].append( [r,c] )      

    def part1(self):
        view_counts = list()
        for pos in self.levels[0]:
            view_counts.append( self.__view_count__(pos) )

        return sum( view_counts )
    
    def part2(self):
        trail_counts = list()
        for pos in self.levels[0]:
            trail_counts.append( self.__trail_count__(pos) )

        return sum( trail_counts )

    def __view_count__( self, pos ):
        # look up or find views from pos
        views, _ = self.__find_views_and_trails__(pos)
        return len(  views  )
    
    def __trail_count__( self, pos ):
        _, trails = self.__find_views_and_trails__(pos)
        return trails

    
    def __find_views_and_trails__(self, pos, height=None ):
        # return the level-9 views reachable from pos by rising 1 level each step
        r,c = pos
        views = self.pos_info[r][c]["views"]
        trails = self.pos_info[r][c]["trails"]

        # return this if we've alredy computed this value earlier
        if views is not None:
            dprint("Using ", pos, "prev values:", views, "and", trails)
            return views, trails

        # lookup height if it isn't supplied
        if height is None:
            height = self.__get_height_from_pos__( pos )
        dprint(pos, height)

        
        # base case: We're already at the view
        if height == 9:
            self.pos_info[r][c]["views"] = [pos]
            self.pos_info[r][c]["trails"] = 1
            return self.pos_info[r][c]["views"], self.pos_info[r][c]["trails"]
         
        # recursion: look at neighbors to develop the answer
        # importantly, we only need neighbors that are hegith h+1, so we avoid inf recursion
        views = list()
        trails = 0
        nbrs = self.__get_neighbors__( pos )
        dprint("nbrs:", nbrs)


        for n in nbrs:
            h = self.__get_height_from_pos__( n )
            if h == height + 1:
                dprint("checking", n )
                # we can get from pos to this n, so get the number of views accessible from there
                iter_views, iter_trails = self.__find_views_and_trails__( n, h )
                # maintain uniqueness for views!
                for iv in iter_views:
                    if iv not in views:
                        views.append(iv)

                dprint(n, "has", iter_views, "views and", iter_trails, "trails" )


                trails += iter_trails

        self.pos_info[r][c]["views"] = views
        self.pos_info[r][c]["trails"] = trails

        return self.pos_info[r][c]["views"], self.pos_info[r][c]["trails"]

    def __get_height_from_pos__( self, pos ):
        return self.pos_info[pos[0]][pos[1]]["height"]
    
    def __get_neighbors__( self, pos ):
        # get the neighbors of pos that are in bounds
        r = pos[0]
        c = pos[1]
        nbr = list()

        if r > 0:
            nbr.append( [r-1, c] )
        if r < self.nr - 1:
            nbr.append( [r+1, c] )
        if c > 0:
            nbr.append( [r, c-1] )
        if c < self.nc - 1:
            nbr.append( [r, c+1] )

        return nbr




In [157]:
test = TopoMap(test_str)

In [158]:
DEBUG = False
test.part1()

36

In [159]:
puzz = TopoMap(puzz_str)
puzz.part1()

430

# Part 2

In [160]:
test.part2()

81

In [161]:
puzz.part2()

928